# **Carregar conjunto de dados**

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

df = pd.read_excel('/Will - Rotulados ELECTRE TRI-rC.xlsx')
df.set_index('Suppliers', inplace=True)
display(df.head())  # Mostra as primeiras linhas do DataFrame



X_criterios = df.iloc[:, 0:6].values

X_criterios

Y_classe = df.iloc[:, 6].values

Y_classe

# **Treinando modelo final**

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_final = GaussianNB(var_smoothing = 0.657933224657568)
nb_final.fit(X_criterios, Y_classe)

In [ ]:

from sklearn.metrics import f1_score, make_scorer, accuracy_score

# Previsões
previsao = nb_final.predict(X_criterios)

# Calculando a acurácia (não necessário para o f1, mas deixado para referência)
accuracy = accuracy_score(Y_classe, previsao)
print("Acurácia:", accuracy)


# **SHAP**

In [ ]:
!pip install shap

In [ ]:
from sklearn.svm import SVC
import numpy as np
import shap


In [ ]:
nb_final.predict_proba(X_criterios)[0]

In [ ]:
nb_final.predict(X_criterios)[0]

In [ ]:
# Nomes dos novos features
feature_names = ['Critério 1', 'Critério 2', 'Critério 3', 'Critério 4', 'Critério 5', 'Critério 6']


In [ ]:
# Get SHAP values
explainer = shap.Explainer(nb_final.predict_proba, X_criterios)
shap_values_nb = explainer(X_criterios)

In [ ]:
# Atualize os valores SHAP com os novos nomes
shap_values_nb.feature_names = feature_names

In [ ]:
nb_final.predict(X_criterios)[10]

In [ ]:
# Waterfall plot for first observation
shap.plots.waterfall(shap_values_nb[16, :, 0])

# Waterfall plot for first observation
shap.plots.waterfall(shap_values_nb[16, :, 1])

# Waterfall plot for first observation
shap.plots.waterfall(shap_values_nb[16, :, 2])

In [ ]:
import pandas as pd

# Supondo que shap_values_nb seja o objeto contendo os valores SHAP
# Extrair os valores SHAP para a 26ª observação (índice 25) para as três classes
shap_values_class_0 = shap_values_nb[25, :, 0]
shap_values_class_1 = shap_values_nb[25, :, 1]
shap_values_class_2 = shap_values_nb[25, :, 2]



# Criar DataFrames para cada classe
df_shap_class_0 = pd.DataFrame({'Feature': feature_names, 'SHAP Value': shap_values_class_0.values})
df_shap_class_1 = pd.DataFrame({'Feature': feature_names, 'SHAP Value': shap_values_class_1.values})
df_shap_class_2 = pd.DataFrame({'Feature': feature_names, 'SHAP Value': shap_values_class_2.values})

# Exibir os DataFrames
print("SHAP values for class 0:")
print(df_shap_class_0)

print("\nSHAP values for class 1:")
print(df_shap_class_1)

print("\nSHAP values for class 2:")
print(df_shap_class_2)


In [ ]:
# Número de fornecedores e classes
num_fornecedores = 50
num_classes = 3
num_criterios = shap_values_nb.shape[1]

# Nome das features (supondo que você tenha uma lista com os nomes das features)
feature_names = [f'Critério {i+1}' for i in range(num_criterios)]

# Criar uma lista para armazenar os dados
dados = []

# Iterar sobre cada fornecedor
for i in range(num_fornecedores):
    for c in range(num_classes):
        shap_values = shap_values_nb[i, :, c].values
        dados.append([f'Fornecedor {i+1}', f'Classe {c}'] + list(shap_values))

# Criar o DataFrame
colunas = ['Fornecedor', 'Classe'] + feature_names
df_shap_values = pd.DataFrame(dados, columns=colunas)

# Exibir o DataFrame
print(df_shap_values)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Especificar o caminho no Google Drive onde você quer salvar o arquivo
file_path = '/content/drive/My Drive/shap_values_fornecedores.csv'

# Salvar o DataFrame em um arquivo CSV no Google Drive
df_shap_values.to_csv(file_path, index=False)

print(f"Arquivo salvo em: {file_path}")



In [ ]:
df_shap_values.to_csv('/mnt/data/shap_values_fornecedores.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Supondo que shap_values_svm.values seja uma matriz numpy com as contribuições SHAP
# e X_criteria.shape[1] é o número de características

# Cálculo das médias dos valores SHAP para cada classe
mean_0 = np.mean(np.abs(shap_values_nb.values[:, :, 0]), axis=0)
mean_1 = np.mean(np.abs(shap_values_nb.values[:, :, 1]), axis=0)
mean_2 = np.mean(np.abs(shap_values_nb.values[:, :, 2]), axis=0)

In [ ]:
import numpy as np

# Normalizando os valores
mean_0_normalized = mean_0 / np.sum(mean_0)

# Exibindo os valores normalizados
for i, mean_0_shap_importance in enumerate(mean_0_normalized):
    print(f'Critério {i+1}: {mean_0_shap_importance}')

In [ ]:
import numpy as np

# Normalizando os valores
mean_1_normalized = mean_1 / np.sum(mean_1)

# Exibindo os valores normalizados
for i, mean_1_shap_importance in enumerate(mean_1_normalized):
      print(f'Critério {i+1}: {mean_1_shap_importance}')

In [ ]:
import numpy as np

# Normalizando os valores
mean_2_normalized = mean_2 / np.sum(mean_2)

# Exibindo os valores normalizados
for i, mean_2_shap_importance in enumerate(mean_2_normalized):
      print(f'Critério {i+1}: {mean_2_shap_importance}')

In [ ]:

# Criação do DataFrame para plotagem
df = pd.DataFrame({"Baixo": mean_0, "Médio": mean_1, "Alto": mean_2})

# Plotagem dos valores SHAP
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
df.plot.bar(ax=ax)

# Configuração dos rótulos dos eixos
ax.set_ylabel('SHAP Médio', size=30)
criteria_labels = [f'Critério {i+1}' for i in range(X_criterios.shape[1])]
ax.set_xticklabels(criteria_labels, rotation=45, size=20)
ax.legend(fontsize=30)

plt.show()

In [ ]:
# Criação do DataFrame para plotagem
df = pd.DataFrame({"Baixo": mean_0, "Médio": mean_1, "Alto": mean_2})

# Plotagem dos valores SHAP
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
df.plot.bar(ax=ax)

# Configuração dos rótulos dos eixos
ax.set_ylabel('SHAP Médio', size=30)
criteria_labels = [f'Critério {i+1}' for i in range(df.shape[0])]
ax.set_xticklabels(criteria_labels, rotation=45, size=20)
ax.legend(fontsize=30)

# Adicionar texto acima das barras
def adicionar_valores_barras(ax, df):
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.2f'),
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha = 'center', va = 'center',
                    xytext = (0, 10),
                    textcoords = 'offset points',
                    fontsize=20)

adicionar_valores_barras(ax, df)

plt.show()

In [ ]:
# Criação do DataFrame para plotagem
df = pd.DataFrame({"Baixo": mean_0_normalized, "Médio": mean_1_normalized, "Alto": mean_2_normalized})

# Plotagem dos valores SHAP
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
df.plot.bar(ax=ax)

# Configuração dos rótulos dos eixos
ax.set_ylabel('SHAP Médio', size=30)
criteria_labels = [f'Critério {i+1}' for i in range(df.shape[0])]
ax.set_xticklabels(criteria_labels, rotation=45, size=20)
ax.legend(fontsize=30)

# Adicionar texto acima das barras
def adicionar_valores_barras(ax, df):
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.2f'),
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha = 'center', va = 'center',
                    xytext = (0, 10),
                    textcoords = 'offset points',
                    fontsize=20)

adicionar_valores_barras(ax, df)

plt.show()

In [ ]:
# get model predictions
preds = nb_final.predict(X_criterios)

new_shap_values = []
for i, pred in enumerate(preds):
    # get shap values for predicted class
    new_shap_values.append(shap_values_nb.values[i][:,pred])

In [ ]:
# replace shap values
shap_values_nb.values = np.array(new_shap_values)
print(shap_values_nb.shape)

In [ ]:
# Definindo os nomes das características
feature_names = ['criterio1', 'criterio2', 'criterio3', 'criterio4', 'criterio5', 'criterio6']

In [ ]:
# Plotando os valores SHAP
shap.summary_plot(shap_values_nb, X_criterios, feature_names=feature_names)


In [ ]:
# Plotando o resumo dos valores SHAP como um gráfico de barras
shap.summary_plot(shap_values_nb, X_criterios, feature_names=feature_names, plot_type="bar")

In [ ]:
import numpy as np
import pandas as pd

# Calculando a importância média de cada característica
shap_values_mean = np.mean(np.abs(shap_values_nb.values), axis=0)

# Agora você pode continuar com suas análises ou visualizações
print(shap_values_mean)

In [ ]:
# Exibindo os valores de shap_values_mean
for i, shap_importance in enumerate(shap_values_mean):
    print(f'Critério {i+1}: {shap_importance}')

In [ ]:
import numpy as np

# Normalizando os valores
shap_values_normalized = shap_values_mean / np.sum(shap_values_mean)

# Exibindo os valores normalizados
for i, shap_importance in enumerate(shap_values_normalized):
    print(f'Critério {i+1}: {shap_importance}')

# **Treinando DT**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Ajustar o modelo da árvore de decisão
dt_final = DecisionTreeClassifier(
    criterion='gini',
    max_depth=None,
    max_features=None,
    min_samples_leaf=2,
    min_samples_split=2,
    splitter='random',
    random_state=0
)
dt_final.fit(X_criterios, Y_classe)


In [ ]:
# Converter os nomes das classes para strings
class_names = [str(cls) for cls in dt_final.classes_]

# Visualizar a árvore de decisão
plt.figure(figsize=(20,10))
plot_tree(dt_final, filled=True, feature_names=df.columns[:6], class_names=class_names)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Fazer previsões com o modelo ajustado
Y_pred = dt_final.predict(X_criterios)

# Calcular a acurácia
accuracy = accuracy_score(Y_classe, Y_pred)
print(f'Acurácia: {accuracy:.2f}')

# Calcular o macro F1-score
macro_f1 = f1_score(Y_classe, Y_pred, average='macro')
print(f'Macro F1-score: {macro_f1:.2f}')